# Extração de distâncias

In [ ]:
import requests
import pandas as pd
import json
import time

# Parâmetros
PROFILE = "driving-hgv"
URL_ROUTE = f"http://localhost:8080/ors/v2/directions/{PROFILE}/geojson"

ORIGENS = ["2611606", "2927408"] # Recife, Salvador
'''
PIB_LIMIAR = 300000  # R$ 300 mil
DAILY_CAP = 1900      # margem abaixo de 2.000
SLEEP_S = 1.6         # ~40/min
'''
# Mapa nome -> [lon, lat]
coords = {
    r["cod_completo"]: {
        "nome_cidade": r["nome_cidade"],
        "lon": float(r["lon"]),
        "lat": float(r["lat"])
    }
    for _, r in df_coordenadas.iterrows()
}

# Garante que as origens estão presentes na tabela
for o in ORIGENS:
    if o not in df_coordenadas['cod_completo'].values:
        raise ValueError(f"Origem ausente na tabela: {o}")

import numpy as np

resultados = []

session = requests.Session()
session.headers.update({"Content-Type": "application/json"})

def consulta_rota(origem_cod, destino_cod):
    o = coords.get(origem_cod)
    o = [o["lon"], o["lat"]]
    d = coords.get(destino_cod)
    d = [d["lon"], d["lat"]]
    '''
    if not o or not d:
        return None, None
    '''
    body = {"coordinates": [o, d]}
    r = session.post(URL_ROUTE, json=body, timeout=60)
    '''
    # Reação simples a 429: aguarda e tenta 1x
    if r.status_code == 429:
        time.sleep(10)
        r = session.post(URL_ROUTE, json=body, timeout=60)
    '''
    if r.status_code != 200:
        print(f"[ERRO {r.status_code}] {origem_cod} -> {destino_cod}: {r.text[:200]}")
        return None, None
    
    data = r.json()

    try:
        summary = data['features'][0]['properties']['summary']
        dist_km = float(summary["distance"]) / 1000.0
        dur_h = float(summary["duration"]) / 3600.0
        resumo = {
            "distancia_km": round(dist_km, 1),
            "duracao_h": round(dur_h, 2),
        }
    except (KeyError, IndexError, TypeError, ValueError):
        resumo = None
    return resumo

#df_coordenadas_filtradas = df_coordenadas.head(10)

for _, cidade in df_coordenadas.iterrows():

    cod_cidade = cidade["cod_completo"]

    for origem_cod in ORIGENS:
        resumo = [origem_cod, cod_cidade, consulta_rota(origem_cod, cod_cidade)]
        resultados.append(resumo)

# Transformar resultados em DataFrame para salvar corretamente
rows = []
for origem, destino, resumo in resultados:
    if resumo is not None and isinstance(resumo, dict):
        distancia_km = resumo.get("distancia_km")
        duracao_h = resumo.get("duracao_h")
    else:
        distancia_km = None
        duracao_h = None
    rows.append([origem, destino, distancia_km, duracao_h])

df_resultados = pd.DataFrame(rows, columns=["origem", "destino", "distancia_km", "duracao_h"])
df_resultados.to_csv('rotas.csv', index=False)

print(f"✅ Feito. Rotas: {len(resultados)}")

# Test Local ORS Server

In [ ]:
import requests

def testar_ors():
    url = "http://localhost:8080/ors/v2/directions/driving-hgv"

    #lon - lat
    params = {
        "start": "-34.88894194457772, -8.062762483052408",
        "end": "-47.50666459311193, -4.95137700016248"
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        dados = response.json()

        if "routes" in dados:
            print("Roteamento OK! Rotas encontradas:", len(dados["routes"]))
        else:
            print("Resposta recebida mas sem rotas:", dados)
    except requests.exceptions.RequestException as e:
        print("Erro na requisição:", e)

if __name__ == "__main__":
    testar_ors()


# Limpezas

## Clean-up lista-cidades.csv

In [2]:
import pandas as pd

FILE = 'raw_data/DTB_2024/RELATORIO_DTB_BRASIL_2024_MUNICIPIOS.xls'
UF_NORDESTE = ['Alagoas', 'Bahia', 'Ceará', 'Maranhão', 'Paraíba', 'Pernambuco', 'Piauí', 'Rio Grande do Norte', 'Sergipe']

df_cod_cidades = pd.read_excel(FILE)

# 2) Remover as 6 primeiras linhas (Table.Skip(...,6))
df_cod_cidades = df_cod_cidades.iloc[5:].reset_index(drop=True)

# 3) Promover a primeira linha a cabeçalho (Table.PromoteHeaders)
df_cod_cidades.columns = df_cod_cidades.iloc[0].astype(str).str.strip()
df_cod_cidades = df_cod_cidades.iloc[1:].reset_index(drop=True)

# 4) Altera tipos de dados
#df_cod_cidades['Município'] = df_cod_cidades['Município'].astype(int)
#df_cod_cidades['UF'] = df_cod_cidades['UF'].astype(int)

# 4) Remover colunas desnecessárias (Table.RemoveColumns)
df_cod_cidades = df_cod_cidades[['UF', 'Nome_UF', 'Município', 'Nome_Município', 'Código Município Completo']]

# 5) Renomear colunas (Table.RenameColumns)
df_cod_cidades.columns = ['cod_uf', 'nome_uf', 'cod_cidade', 'nome_cidade', 'cod_completo']


df_cod_cidades_filtrado = df_cod_cidades[df_cod_cidades['nome_uf'].isin(UF_NORDESTE)].reset_index(drop=True)

## Clean-up Coordenadas

In [3]:
import pandas as pd

FILE = 'raw_data/anexo_16261_Coordenadas_Sedes_5565_Municípios_2010.xls'

df_coordenadas = pd.read_excel(FILE, dtype=str)

# 1) Renomear colunas (Table.RenameColumns)
df_coordenadas.columns = ['cod_completo', 'nome_cidade', 'lon', 'lat']

# 2) Filtrar apenas as cidades do Nordeste

df_coordenadas = df_coordenadas[df_coordenadas['cod_completo'].isin(df_cod_cidades_filtrado['cod_completo'])]

#print(df_coordenadas.head(10))

## Clean-up PIB

In [4]:
import pandas as pd

UF_NORDESTE = ['Alagoas', 'Bahia', 'Ceará', 'Maranhão', 'Paraíba', 'Pernambuco', 'Piauí', 'Rio Grande do Norte', 'Sergipe']

df_pib = pd.read_excel('raw_data/PIB dos Municípios - base de dados 2010-2021.xlsx', dtype=str)

# 1) Remover colunas desnecessárias (Table.RemoveColumns)
df_pib = df_pib[['Ano', 'Código da Unidade da Federação', 'Nome da Unidade da Federação', 'Código do Município', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)']]

# 2) Renomear colunas (Table.RenameColumns)
df_pib.columns = ['ano', 'cod_uf', 'nome_uf', 'cod_completo', 'pib', 'pib_per_capita']

# 3) Filtrar apenas as cidades do Nordeste
df_pib = df_pib[df_pib['nome_uf'].isin(UF_NORDESTE)].reset_index(drop=True)

# 4) Filtrar apenas o ano de 2021
df_pib = df_pib[df_pib['ano'] == '2021'].reset_index(drop=True)

# 5) Ajustar tipos de dados
df_pib['pib'] = df_pib['pib'].astype(float) #.str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)
df_pib['pib_per_capita'] = df_pib['pib_per_capita'].astype(float) #.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

# Construção dos grafo v1

## Preparação dados cidades

In [ ]:
import pandas as pd

df_distancias = pd.read_csv('raw_data/rotas.csv', dtype=str)
df_distancias['distancia_km'] = df_distancias['distancia_km'].astype(float)
df_distancias['duracao_h'] = df_distancias['duracao_h'].astype(float)

df_distancias = df_distancias[df_distancias['distancia_km'].notnull()].reset_index(drop=True)

print(df_distancias.head(10))

cidades = df_cod_cidades_filtrado.merge(df_pib[['cod_completo', 'pib']], on='cod_completo', how='left')

cidades = cidades[['cod_completo','nome_cidade','pib']]

cidades = cidades.merge(df_coordenadas[['cod_completo', 'lon', 'lat']], on='cod_completo', how='left')
cidades['lat'] = cidades['lat'].astype(float)
cidades['lon'] = cidades['lon'].astype(float)

#print(cidades.head(10))


## Visualização do Grafo

In [ ]:
'''
import pandas as pd
from pyvis.network import Network

# Suponha ORIGEM com os códigos de Recife e Salvador
ORIGEM = ['2611606', '2927408']  # Exemplo códigos Recife e Salvador

# Limitar a 100 cidades para visualização
#cidades_filtradas = cidades.head(100)

# Garantir que Recife e Salvador estejam na lista de nós
for codigo in ORIGEM:
    if codigo not in cidades['cod_completo'].values:
        row = cidades[cidades['cod_completo'] == codigo]
        if not row.empty:
            cidades = pd.concat([cidades, row], ignore_index=True)

# Lista atualizada dos códigos existentes no grafo
codigos_filtrados = cidades['cod_completo'].tolist()

# Filtrar arestas para garantir que origem e destino existam nos nós
df_distancias_filtrado = df_distancias[
    df_distancias['origem'].isin(codigos_filtrados) & df_distancias['destino'].isin(codigos_filtrados)
]

# Criar o grafo pyvis
net = Network(height='1080px', width='1920px', bgcolor='#222222', font_color='white')

# Normalizar uma coluna do DataFrame para intervalo [0, 1000]
def normalize(series, new_min=0, new_max=10000):
    min_val = series.min()
    max_val = series.max()
    return ((series - min_val) / (max_val - min_val)) * (new_max - new_min) + new_min

# Normaliza lon e lat do DataFrame
cidades['x'] = normalize(cidades['lon'])
cidades['y'] = normalize(-cidades['lat'])  # invertendo lat para visualização

# Usar cidades_filtradas['x'] e ['y'] no loop de adição dos nós
for _, row in cidades.iterrows():
    cor = 'red' if row['cod_completo'] in ORIGEM else 'blue'
    tamanho = 50 if row['cod_completo'] in ORIGEM else 20
    titulo = f"{row['nome_cidade']}<br>PIB: {row['pib']}"
    net.add_node(
        row['cod_completo'],
        label=row['nome_cidade'],
        title=titulo,
        font={'size':1},
        color=cor,
        size=tamanho,
        x=row['x'],
        y=row['y'],
        fixed=True
    )

# Adicionar as arestas com distância como label
for _, row in df_distancias_filtrado.iterrows():
    net.add_edge(row['origem'], row['destino'], label=f"{row['distancia_km']} km", color='green', font={'size': 2})

# Desliga a física para evitar movimentação dos nós
net.toggle_physics(False)

# Salvar arquivo e mostrar grafo
net.save_graph('grafo_cidades.html')
'''

In [ ]:
import pandas as pd
from pyvis.network import Network

# Suponha que os DataFrames 'cidades' e 'df_distancias' já estejam carregados corretamente

# Códigos das cidades-centro (Recife e Salvador)
ORIGEM = ['2611606', '2927408']

# Função para normalizar uma coluna
def normalize(series, new_min=0, new_max=10000):
    min_val = series.min()
    max_val = series.max()
    return ((series - min_val) / (max_val - min_val)) * (new_max - new_min) + new_min

def get_edge_color(dist, dist_min, dist_max):
    # Normaliza distância para intervalo [0,1]
    norm = (dist - dist_min) / (dist_max - dist_min)
    # Gerar cor RGB: perto = verde, longe = vermelho
    red = int(norm * 255)
    green = int((1 - norm) * 255)
    return f'rgb({red},{green},0)'

# Normaliza as coordenadas das cidades para o plano da visualização
cidades['x'] = normalize(cidades['lon'])
cidades['y'] = normalize(-cidades['lat'])  # inverte latitude para melhor distribuição

# Normaliza o PIB para definir o tamanho dos nós
cidades['pib_normalizado'] = normalize(cidades['pib'], new_min=10, new_max=100)

# Normaliza a distancia para definir a cor das arestas
#cidades['distancia_normalizada'] = normalize(cidades['pib'], new_min=1, new_max=10)
dist_min = df_distancias['distancia_km'].min()
dist_max = df_distancias['distancia_km'].max()

# Cria o grafo pyvis (visualização grande)
net = Network(height='1080px', width='1920px', bgcolor='#222222', font_color='white')

# Adiciona os nós ao grafo
for _, row in cidades.iterrows():
    cor = 'red' if row['cod_completo'] in ORIGEM else 'blue'
    tamanho = row['pib_normalizado'] #retorna_tamanho(row)
    titulo = f"{row['nome_cidade']}<br>PIB: {row['pib']}"
    net.add_node(
        row['cod_completo'],
        label=row['nome_cidade'],
        title=titulo,
        color=cor,
        size=tamanho,
        font={'size': 12},  # fonte legível
        x=row['x'],
        y=row['y'],
        fixed=True
    )

# Adiciona as arestas ao grafo
for _, row in df_distancias.iterrows():
    color = get_edge_color(row['distancia_km'], dist_min, dist_max)
    net.add_edge(
        row['origem'],
        row['destino'],
        label=f"{row['distancia_km']} km",
        color=color,
        font={'size': 10}  # tamanho da fonte das distâncias
    )

# Mantém os nós fixos nas coordenadas, sem física dinâmica
net.toggle_physics(False)

# Salva e gera o grafo interativo em HTML
net.save_graph('grafo_cidades.html')


# Construção grafo v2

## Construção Grafo

### Preapara Dados

In [5]:
import pandas as pd

df_distancias = pd.read_csv('raw_data/rotas.csv', dtype=str)
df_distancias['distancia_km'] = df_distancias['distancia_km'].astype(float)
df_distancias['duracao_h'] = df_distancias['duracao_h'].astype(float)

df_distancias = df_distancias[df_distancias['distancia_km'].notnull()].reset_index(drop=True)

#print(df_distancias.head(10))

cidades = df_cod_cidades_filtrado.merge(df_pib[['cod_completo', 'pib']], on='cod_completo', how='left')

cidades = cidades[['cod_completo','nome_cidade','pib']]

cidades = cidades.merge(df_coordenadas[['cod_completo', 'lon', 'lat']], on='cod_completo', how='left')
cidades['lat'] = cidades['lat'].astype(float)
cidades['lon'] = cidades['lon'].astype(float)

#print(cidades.head(10))

### Gera grafo

In [6]:
from matplotlib import pyplot as plt
import networkx as nx

ORIGEM = ['2611606', '2927408']

# Extrair grafo do pyvis para NetworkX (se você criou com pyvis, crie também um NetworkX paralelo)
G = nx.Graph()
for _, row in cidades.iterrows():
    G.add_node(row['cod_completo'], nome=row['nome_cidade'], pib=row['pib'])
for _, row in df_distancias.iterrows():
    G.add_edge(row['origem'], row['destino'], distancia=row['distancia_km'])


'''nx.draw(G,
        with_labels=False,
        node_size=10,
        node_color= 'blue',
        font_size=8,
        font_color='white',
        edge_color='gray')
plt.show()'''

"nx.draw(G,\n        with_labels=False,\n        node_size=10,\n        node_color= 'blue',\n        font_size=8,\n        font_color='white',\n        edge_color='gray')\nplt.show()"

## Realiza análise

In [ ]:
#Grau de cada cidade: Quantos vizinhos cada cidade tem.
graus = dict(G.degree())
sorted_graus = sorted(graus.items(), key=lambda x: x[1], reverse=True)
print(sorted_graus)

In [ ]:
#Qual cidade está mais conectada? (Maior grau)
max_grau_cidade = max(graus, key=graus.get)
print("Mais conectada:", G.nodes[max_grau_cidade]['nome'], "com grau", graus[max_grau_cidade])

In [ ]:
# Caminho mais curto entre Recife e Salvador:
caminho_curto = nx.shortest_path(G, source='2611606', target='2927408', weight='distancia')
print("Caminho mais curto entre Recife e Salvador:", caminho_curto, "com distância total de", nx.shortest_path_length(G, source='2611606', target='2927408', weight='distancia'), "km")

In [ ]:
# Centralidade: Quais cidades estão mais “no centro” da logística?

centralidade = nx.closeness_centrality(G, distance='distancia')
mais_central = max(centralidade, key=centralidade.get)
print('Cidade mais central:', G.nodes[mais_central]['nome'])

#centralidade = nx.betweenness_centrality(G, weight='distancia')
#print("Centralidade:", sorted(centralidade.items(), key=lambda x: x[1], reverse=True)[:10])

In [ ]:
#Explorar interações economicamente
# Relacionar PIB com número de conexões
'''
relacao = [(G.nodes[n]['nome'], G.nodes[n]['pib'], G.degree(n)) for n in G.nodes]
relacao.sort(key=lambda x: x[2], reverse=True)
print(relacao[:10])  # Top 10 cidades mais conectadas com seus PIBs
'''
# detalhamento do cálculo

#Obter o grau (número de conexões) de cada cidade no grafo:
#O grau de um nó na rede indica quantas arestas (conexões) ele tem.
graus = dict(G.degree())

#Extrair o PIB e o nome de cada cidade a partir do grafo:
#Cada nó pode conter atributos (como nome e pib) que guardamos quando criamos o grafo. Podemos construir uma lista com (nome, pib, grau):
relacao = [(G.nodes[n]['nome'], G.nodes[n]['pib'], graus[n]) for n in G.nodes]

#Ordenar a lista pelo grau de conexão para destacar as cidades mais conectadas:
relacao.sort(key=lambda x: x[2], reverse=True)
print(relacao[:10])  # Top 10 cidades mais conectadas com seus PIBs


In [ ]:
import networkx as nx

# Filtrar cidades de maior PIB
top_n = G.nodes.__len__() // 1  # 10=10% 1=100% ou outro valor desejado

top_pib_cidades = sorted([(n, G.nodes[n]['pib']) for n in G.nodes], key=lambda x: x[1], reverse=True)[:top_n]
top_nodes = [n for n, pib in top_pib_cidades]

#print("Cidades com maior PIB:", [(G.nodes[n]['nome'], G.nodes[n]['pib']) for n in top_nodes]) 

# Para Recife
distancias_recife = {}
for cidade in top_nodes:
    try:
        distancia = nx.shortest_path_length(G, source='2611606', target=cidade, weight='distancia')
        distancias_recife[cidade] = distancia
    except nx.NetworkXNoPath:
        distancias_recife[cidade] = float('inf')  # sem caminho

# Para Salvador
distancias_salvador = {}
for cidade in top_nodes:
    try:
        distancia = nx.shortest_path_length(G, source='2927408', target=cidade, weight='distancia')
        distancias_salvador[cidade] = distancia
    except nx.NetworkXNoPath:
        distancias_salvador[cidade] = float('inf')

# Calcular soma ponderada por PIB
def soma_ponderada(dist_dict, G):
    soma = 0
    sem_caminho = []
    for cidade, dist in dist_dict.items():
        if dist == float('inf'):
            sem_caminho.append(cidade)
            continue
        pib = G.nodes[cidade]['pib']
        soma += dist * pib
    return soma, sem_caminho

peso_recife, recife_sem_caminho = soma_ponderada(distancias_recife, G)
peso_salvador, salvador_sem_caminho = soma_ponderada(distancias_salvador, G)

print("Peso logístico ponderado por PIB - Recife:", peso_recife)
#print("Cidades sem caminho para Recife:", recife_sem_caminho)
#print("Peso logístico ponderado por PIB - Salvador:", peso_salvador)
print("Cidades sem caminho para Salvador:", salvador_sem_caminho)
percent = abs(peso_recife - peso_salvador) / min(peso_recife, peso_salvador) * 100
if peso_recife < peso_salvador:
    print(f"Portanto Recife é {percent:.2f}% mais estratégico que Salvador")
else:
    print(f"Portanto Salvador é {percent:.2f}% mais estratégico que Recife")

Peso logístico ponderado por PIB - Recife: 896367544614.3162
Cidades sem caminho para Salvador: ['2605459', '2404101', '2800704']
Portanto Recife é 12.23% mais estratégico que Salvador
